In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib

import ribasim

import ribasim_lumping
# from ribasim_lumping.ribasim_model_results import plot_results_basin_ribasim_model, \
#     plot_results_basins_ribasim_model, get_ribasim_basin_data_from_model, read_ribasim_model_results

import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow as pa

import warnings

warnings.simplefilter("ignore")
pd.options.mode.chained_assignment = None

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# define network name, base dir
network_name            = "tki_zutphen_berkel"
base_dir                = Path("..\\..\\..\\ribasim_lumping_data\\")
# base_dir                = Path("..\\..\\..\\..\\..\\..\\ribasim_lumping_data\\")
# directory D-HYDRO basis calculations
dhydro_basis_dir        = Path(base_dir, network_name, "2_dhydro\\") # zelfde directory, want gebruik zelfde som
# directory D-HYDRO results calculations
dhydro_results_dir      = Path(base_dir, network_name, "2_dhydro\\") # zelfde directory, want gebruik zelfde som
# directory results
results_dir             = Path(base_dir, network_name, "3_ribasim\\")
# batch file used to calculate volume D-Hydro module per GridPoint
dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
dhydro_volume_tool_increment = 0.1
# link to ribasim_executable
path_ribasim_executable = "..\\..\\..\\ribasim_cli_windows\\bin\\ribasim.exe"

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    path_ribasim_executable=path_ribasim_executable,
    crs=28992
)
# network.export_to_geopackage()

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_data", "areas")
areas_file_name         = f"{network_name}_afw_eenheden.shp"
areas_id_column         = "GFEIDENT"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)

In [ ]:
network.plot();

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type='dhydro',
    set_name="winter",
    set_start_month=10,
    set_start_day=1,
    model_dir=Path(dhydro_basis_dir, "afvoergolf"),
    simulation_name="tki_zuthpen_berkel_afvoergolf.dsproj_data",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=True,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
# network.add_basis_network(
#     source_type='dhydro',
#     set_name="winter",
#     set_start_month=10,
#     set_start_day=1,
#     model_dir=Path(dhydro_basis_dir, "afvoergolf"),
#     simulation_name="data",
#     dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
#     dhydro_volume_tool_force=True,
#     dhydro_volume_tool_increment=dhydro_volume_tool_increment
# );
# network.add_basis_network(   
#     source_type='dhydro',
#     set_name='zomer', 
#     set_start_month=4,
#     set_start_day=1,
#     model_dir=Path(dhydro_basis_dir, "zomer"),
#     simulation_name="tki_zuthpen_berkel_basis.dsproj_data",
#     dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
#     dhydro_volume_tool_force=True,
#     dhydro_volume_tool_increment=0.1
# );

In [ ]:
network.plot()

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_simulation_set(
    source_type='dhydro',
    set_name="winter",
    model_dir=Path(dhydro_results_dir, "winter"),
    simulation_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulation_ts=pd.date_range("2000-01-06 23:00", periods=9, freq="6D"),
);
# network.add_simulation_set(
#     source_type='dhydro',
#     set_name="winter",
#     model_dir=Path(dhydro_results_dir, "winter"),
#     simulation_names=["data"],
#     simulation_ts=pd.date_range("2000-01-06 23:00", periods=9, freq="6D"),
# );
# network.add_simulation_data(
#     source_type='dhydro',
#     set_name="zomer",
#     model_dir=Path(dhydro_results_dir, "winter"),
#     simulation_names=["tki_zuthpen_berkel_basis.dsproj_data"],
#     simulation_ts=pd.date_range("2000-01-06 23:00", periods=9, freq="6D"),
# );

In [ ]:
# network.change_branches_direction(
#     branche_ids=[]
# )

#### OPZETTEN SIMULATIE

In [ ]:
simulation_code = "tki_zutphen_berkel"

instellingen (eventueel tijdsafhankelijk)

In [ ]:
# Definition settings changes over time
settings_time_variation = False
# in case of no variation over time. set set_name ("winter", "zomer", ...)
settings_set_name = 'winter'

definitie laterale instroom

In [ ]:
# Defition Lateral inflow
# method_laterals = 1: laterals basis network, 
# method_laterals = 2: laterals per discharge areas, 
# method_laterals = 3: homogeneous discharge
network.method_laterals = 2

# in case of 2: give direction to csv file with timeseries per area -> column names according to areas[areas_id_column]
areas_laterals_path = Path(base_dir, network_name, "1_data", "laterals", "laterals_areas.csv")
network.read_areas_laterals_timeseries(areas_laterals_path, sep=";", dayfirst=True)

definitie randvoorwaarden

In [ ]:
# test boundary data
# method_boundaries = 1: use boundaries basis network, 
# method_boundaries = 2: use timeseries from imported csv, 
# method_boundaries = 3: use fixed boundaries
network.method_boundaries = 2

boundaries_timeseries_path = Path(base_dir, network_name, "1_data", "boundaries", "boundary_timeseries.csv")
network.read_boundaries_timeseries_data(boundaries_timeseries_path, skiprows=1, sep=";")

In [ ]:
network.boundaries_timeseries_data.plot()

In [ ]:
# changes to timeseries data:
# - start and end time equal to laterals
# - backward fill of data from Almen (1389)
network.boundaries_timeseries_data = network.boundaries_timeseries_data.loc[
    network.laterals_areas_data.index[0]:network.laterals_areas_data.index[-1]
].bfill()
network.boundaries_timeseries_data.plot();

definitie intitiële waterstanden

In [ ]:
# Definition initial water levels (refer to simulation name and index timestep)
# method_initial_waterlevels = 1: based on one of the basis simulations
# method_initial_waterlevels = 2: based on one of the simulations provided
# method_initial_waterlevels = 3: based on areas, define a column to be used
network.method_initial_waterlevels = 2

# give name of simulation plus the selected timestep (int)
network.initial_waterlevels_set_name = "tki_zuthpen_berkel_afvoergolf.dsproj_data"
network.initial_waterlevels_timestep = 3

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    orifices=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kdu_DR80770020', # duiker afvoergebiedje noordzijde Berkel
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
        'kdu_DR80740070', # inlaat twentekanaal
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
        # 'kst_ST80830045', # weir tussen uitstroompunten Zutphen
        # 'kst_ST80820031', 'kst_ST80820042', # test
    ], 
    edge_ids_to_include=[1801],
);

In [ ]:
split_node_type_conversion = dict(
    weir="TabulatedRatingCurve",
    universalWeir="TabulatedRatingCurve",
    pump="Pump",
    culvert="TabulatedRatingCurve",
    edge="ManningResistance",
    orifice="Outlet",
    boundary_connection="ManningResistance"
)
# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = dict(
    kdu_DR80740070='ManningResistance',
    # duikers voor wijk Zuidwijken
    kdu_DR80940046='ManningResistance', 
    kdu_DR80950043='ManningResistance', 
    kdu_DR80950151='ManningResistance', 
)

In [ ]:
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);

In [ ]:
ribasim_model = network.generate_ribasim_model_complete(
    # dummy_model=True,
    set_name=settings_set_name,
    saveat=3600,
)